In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalizationV2

In [2]:
train_path = '../data/blur_combined/seg_train/'
test_path = '../data/blur_combined/seg_test/'

In [3]:
train_datagen = ImageDataGenerator(
                    rescale = 1/255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = False)
training_set = train_datagen.flow_from_directory(train_path,
                                                   target_size = (64, 64),
                                                 class_mode = 'categorical')

Found 28068 images belonging to 6 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            class_mode = 'categorical')

Found 6000 images belonging to 6 classes.


In [5]:
cnn = Sequential()
cnn

Metal device set to: Apple M1


2022-08-29 22:17:27.856496: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-29 22:17:27.856620: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Dropout(0.4)),

cnn.add(Flatten())

cnn.add(Dense(units=256, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=128, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=64, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=24, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=6, activation='softmax'))

In [7]:
cnn.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
%%time
cnn.fit(training_set, steps_per_epoch= len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 10)

Epoch 1/10


2022-08-29 22:17:28.349213: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-29 22:17:29.341622: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


878/878 [==============================] - ETA: 0s - loss: 1.3886 - accuracy: 0.4609

2022-08-29 22:18:25.711895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


878/878 [==============================] - 62s 69ms/step - loss: 1.3886 - accuracy: 0.4609 - val_loss: 1.5478 - val_accuracy: 0.3927
Epoch 2/10
878/878 [==============================] - 61s 69ms/step - loss: 1.0362 - accuracy: 0.6026 - val_loss: 0.9067 - val_accuracy: 0.6578
Epoch 3/10
878/878 [==============================] - 66s 75ms/step - loss: 0.9376 - accuracy: 0.6492 - val_loss: 0.8304 - val_accuracy: 0.7017
Epoch 4/10
878/878 [==============================] - 65s 74ms/step - loss: 0.8383 - accuracy: 0.6936 - val_loss: 0.8116 - val_accuracy: 0.7125
Epoch 5/10
878/878 [==============================] - 64s 73ms/step - loss: 0.7825 - accuracy: 0.7196 - val_loss: 0.7718 - val_accuracy: 0.6960
Epoch 6/10
878/878 [==============================] - 59s 67ms/step - loss: 0.7352 - accuracy: 0.7396 - val_loss: 1.1220 - val_accuracy: 0.5737
Epoch 7/10
878/878 [==============================] - 59s 68ms/step - loss: 0.7010 - accuracy: 0.7500 - val_loss: 1.1239 - val_accuracy: 0.5803
Epo

In [10]:
accuracy=cnn.evaluate(test_set, steps=len(test_set),verbose=2)
print('Accuracy of the model on the test set: ',accuracy[1])

188/188 - 5s - loss: 0.6528 - accuracy: 0.7665 - 5s/epoch - 25ms/step
Accuracy of the model on the test set:  0.7664999961853027
